# Introduction


**Note:** Use Jupyter Notebook or Google Colab (NOT Jupyter Lab).

The messages within the datasets have already been classified as spam or ham (not spam).

The implementation of a Naïve Bayes Spam Filter is relatively straightforward without scikit-learn, and this exercise will help you understand the implementation details.

Bayes Theorem can give us the probability that a message is spam \(S\) for a given event \(E\):

$$
P\left(S \mid E\right) = \frac{P\left(E \mid S\right)P\left(S\right)}{P\left(E \mid S\right)P\left(S\right) + P\left(E \mid \lnot S\right)P\left(\lnot S\right)}
$$

Where:

- \(P\left(S \mid E\right)\) is the probability that the message is spam given the event occurred.
- \(P\left(S\right)\) is the prior probability that a message is spam.
- \(P\left(\lnot S\right)\) is the prior probability that a message is not spam.

Note: \(P\left(S\right)\) and \(P\left(\lnot S\right)\) are prior values or beliefs. You can calculate them using the dataset's spam and ham counts, or assume arbitrary values, like 80% of emails are spam and 20% are not. The filter's success depends on these prior values.

- \(P\left(E \mid S\right)\) is the probability that event \(E\) occurs in spam emails.
- \(P\left(E \mid \lnot S\right)\) is the probability that event \(E\) occurs in non-spam emails.


<h3>1.  Read the dataset into a dataframe and explore</h3>
<p>Start by importing pandas and read the dataset into a DataFrame named df.  Output the first 20 rows of the dataframe to get a general feel of how the data is structured.</p>
<p>You may encounter the error: UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 135-136: invalid continuation byte.  You don't need to edit the datafile,  as you should be able to successfully read in the datafile by changing the encoding to latin-1.</p>


In [ ]:
import pandas as pd

# Read the dataset into a DataFrame
df = pd.read_csv('spam.csv', encoding='latin-1')

# Output the first 20 rows of the DataFrame
print(df.head(20))

<h3>2. Clean the data</h3>
<p>We are only interested in words, clean the data so that all punctuations are removed.  You should be left with a dataset that only contains alpha characters (including spaces).  You should also ensure all the words are lowercase.  Store the cleaned data into a DataFrame named clean.</p>

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Category</th>
      <th>Message</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ham</td>
      <td>go until jurong point crazy available only in ...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ham</td>
      <td>ok lar joking wif u oni</td>
    </tr>
    <tr>
      <th>2</th>
      <td>spam</td>
      <td>free entry in  a wkly comp to win fa cup final...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ham</td>
      <td>u dun say so early hor u c already then say</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ham</td>
      <td>nah i dont think he goes to usf he lives aroun...</td>
    </tr>
  </tbody>
</table>



In [ ]:
import pandas as pd
import re

# Assuming 'df' contains the DataFrame with the original data
# Create a copy of the DataFrame to work with
clean = df.copy()

# Remove punctuations and convert to lowercase
clean['v2'] = clean['v2'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x).lower())

# Output the first 20 rows of the cleaned DataFrame
print(clean.head(20))

<h3>3. Split the Data</h3>
<p>Split the data into three random samples, one for training the model, one for validation and the other for testing the model.  Create DataFrames named train_data, validation_data and test_data.  The train_data DataFrame should contain 60-70% of the data, validation_data 15-20% and the test_data DataFrame the remaining data.<p>  


In [ ]:
import pandas as pd
import numpy as np

# Assuming 'clean' is your DataFrame containing the cleaned data
# Shuffle the DataFrame
clean = clean.sample(frac=1).reset_index(drop=True)

# Determine the proportion of each dataset
train_percent = 0.7  # 70% of data for training
validation_percent = 0.15  # 15% of data for validation

# Calculate split indices
train_end = int(train_percent * len(clean))
validation_end = train_end + int(validation_percent * len(clean))

# Split the DataFrame
train_data = clean[:train_end]
validation_data = clean[train_end:validation_end]
test_data = clean[validation_end:]

# Output the number of entries in each set
print(f"Training Set: {len(train_data)} entries, {100 * len(train_data) / len(clean):.2f}% of total")
print(f"Validation Set: {len(validation_data)} entries, {100 * len(validation_data) / len(clean):.2f}% of total")
print(f"Test Set: {len(test_data)} entries, {100 * len(test_data) / len(clean):.2f}% of total")


<h3>4. Create a Word Frequency DataFrame</h3>
<p>Create a new DataFrame named word_freq that contains each word with the number of times it appears in a spam and a ham message.  You should use the train_data.</p>
<p>Below is an example of what the DataFrame would look like, <i>note</i> that your values may differ depending on how the data was split.</p>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Word</th>
      <th>#Spam</th>
      <th>#Ham</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>0</td>
      <td>go</td>
      <td>27</td>
      <td>196</td>
    </tr>
    <tr>
      <td>1</td>
      <td>until</td>
      <td>4</td>
      <td>17</td>
    </tr>
    <tr>
      <td>2</td>
      <td>jurong</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>3</td>
      <td>point</td>
      <td>1</td>
      <td>9</td>
    </tr>
    <tr>
      <td>4</td>
      <td>crazy</td>
      <td>4</td>
      <td>8</td>
    </tr>
    <tr>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <td>7253</td>
      <td>salesman</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>7254</td>
      <td>pity</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>7255</td>
      <td>soany</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>7256</td>
      <td>suggestions</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>7257</td>
      <td>bitching</td>
      <td>1</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


In [ ]:
import pandas as pd

word_freq = {}

for index, row in train_data.iterrows():
    words = row['v2'].split()
    label = row['v1']
    for word in words:
        if word not in word_freq:
            word_freq[word] = {'spam_count': 0, 'ham_count': 0}
        word_freq[word][f'{label}_count'] += 1

# Create a DataFrame from the word frequency dictionary
word_freq_df = pd.DataFrame(word_freq).transpose().fillna(0)

# Output the first few rows of the word_freq DataFrame
print(word_freq_df.head())

<h3>5. Visualise the Data</h3>
<p>Let's use a Word Cloud library to visualise the most common words contained in spam messages.</p>

[Example of a Word Cloud Image](https://drive.google.com/open?id=1lVRGHtMB1AMJf-JSi7MmcHbZB_BvBhGC)





In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate a string containing all words in spam messages
spam_words = ' '.join(train_data[train_data['v1'] == 'spam']['v2'])

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='black').generate(spam_words)

# Plot word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud for Spam Messages')
plt.axis('off')
plt.show()

<h3>6.  Calculate $P\left(E\middle| S\right)$ and $P\left(E|\lnot S\right)$</h3>
<p>Next create a new DataFrame named word_prob that gives the probability of each word being found in a spam and ham message.</p>
<p>To calculate the probability of a word being spam you divide the number of times the word was found in spam by the total number of spam messages, likewise to calculate the probability of each word being found in a ham message you divide the number of times the word was found in a ham message by the total number of ham messages.</p>
<p>If a word was not found in ham or spam it will cause problems later because the probability calculated will be zero. Therefore, use a pseudocount k and estimate the probability of seeing the word. This is known as smoothing and results in the following formula when k = 0.5, for example.</p>
<p>$P\left(E\middle| S\right)$ = (number of spams containing the word + k) / (total number of spam messages + 2 * k).</p>
<p>Likewise, for $P\left(E|\lnot S\right)$.</p>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Word</th>
      <th>P(E|S)</th>
      <th>P(E|¬S)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>go</td>
      <td>0.053322</td>
      <td>0.050055</td>
    </tr>
    <tr>
      <th>1</th>
      <td>until</td>
      <td>0.011364</td>
      <td>0.004275</td>
    </tr>
    <tr>
      <th>2</th>
      <td>jurong</td>
      <td>0.002622</td>
      <td>0.000138</td>
    </tr>
    <tr>
      <th>3</th>
      <td>point</td>
      <td>0.002622</td>
      <td>0.002344</td>
    </tr>
    <tr>
      <th>4</th>
      <td>crazy</td>
      <td>0.011364</td>
      <td>0.002344</td>
    </tr>
  </tbody>
</table>



In [ ]:
import pandas as pd

# Pseudocount k
k = 0.5

# Total number of spam messages
total_spam_messages = len(train_data[train_data['v1'] == 'spam'])

# Total number of ham messages
total_ham_messages = len(train_data[train_data['v1'] == 'ham'])

# Initialize an empty dictionary to store word probabilities
word_prob = {}

# Iterate through each word in word_freq_df
for word in word_freq_df.index:
    # Calculate P(E|S)
    spam_count = word_freq_df.loc[word, 'spam_count']
    prob_word_given_spam = (spam_count + k) / (total_spam_messages + 2 * k)
    
    # Calculate P(E|¬S)
    ham_count = word_freq_df.loc[word, 'ham_count']
    prob_word_given_ham = (ham_count + k) / (total_ham_messages + 2 * k)
    
    # Store probabilities in the word_prob dictionary
    word_prob[word] = {'prob_spam': prob_word_given_spam, 'prob_ham': prob_word_given_ham}

# Create a DataFrame from the word_prob dictionary
word_prob_df = pd.DataFrame(word_prob).transpose()

# Output the first few rows of the word_prob DataFrame
print(word_prob_df.head())

<h3>7. Checking the 'spamliness' of a single word</h3>
<p>Now that we have trained the model, we will test the model.  Before we use the test_data, first let’s check how the model calculates the spamliness of a single word.  This is where we use the Bayes Theorem formula.  We have already calculated $P\left(E\middle| S\right)$ and $P\left(E|\lnot S\right)$, so we can just extract these values from the word_prob DataFrame.</p>
<p>We need to decide on the prior values $P\left(S\right)$ and $P\left(\lnot S\right)$, this is where you can experiment and tweak the model, in this example the prior value for spam was set to $0.4$ and the prior value for not spam or ham was set to $0.6$.</p>
<h3>
$P\left(S\middle|\ E\right)=\frac{P\left(E\middle|\ S\right)P\left(S\right)}{P\left(E\middle|\ S\right)P\left(S\right)+P\left(E|\lnot S\right)P\left(\lnot S\right)}$
</h3>
<pre>
Output
Word = ['free']
P(E|S) = [0.29108392]
P(E|¬S) = [0.01365141]
P(S|E) = [0.93427577]
P(¬S|E) = [0.06572423]
</pre>



In [ ]:
# Prior probabilities
prior_spam = 0.4
prior_ham = 0.6

# Function to calculate spamliness of a single word
def calculate_spamliness(word):
    # Retrieve P(E|S) and P(E|¬S) from word_prob_df
    prob_word_given_spam = word_prob_df.loc[word, 'prob_spam']
    prob_word_given_ham = word_prob_df.loc[word, 'prob_ham']
    
    # Calculate P(S|E) using Bayes' theorem formula
    prob_spam_given_word = (prob_word_given_spam * prior_spam) / \
                           ((prob_word_given_spam * prior_spam) + (prob_word_given_ham * (1 - prior_spam)))
    
    return prob_spam_given_word

# Example word to check
word_to_check = 'offer'

# Calculate spamliness of the word
spamliness = calculate_spamliness(word_to_check)

# Output the spamliness of the word
print(f"The spamliness of the word '{word_to_check}' is: {spamliness}")


<h3>8. Checking the 'spamliness' of several words</h3>
<p>To check the spamliness of several words contained in a message we multiply the probabilities.  The model assumes the words appear as independent events hence the naïve Bayes.  In reality of course, words are not independent events, but the model still performs well.  So we use the assumption that the words appear independently, and hence we multiply probabilities, so
$P(S\,|\, x_1,\dots,x_n)\approx \frac{P(S)\underset{i=1}{\overset{n}{\prod}}P(x_i | S)}{P(S)\underset{i=1}{\overset{n}{\prod}}P(x_i | S)+P(\neg S)\underset{i=1}{\overset{n}{\prod}}P(x_i | \neg S)}$

Calculate the probability for each word in a message being spam, you might want to store the calculations in a list named prob_spam.  Likewise create a list for each word not being spam.
Then multiply the probabilities and compare the results.  If the result of multiplying the probabilities for spam is greater than the result of multiplying the probabilities for not spam, then you assume the message as spam.
</p>
<p>If you have a word in your message that is not in the word_prob DataFrame then you can't get the probability.  Skip any words in the message that are not in the word_prob DataFrame.</p>


In [ ]:
import numpy as np

def calculate_message_spamliness(message):
    words = message.split()
    
    prob_spam = []
    prob_not_spam = []
    
    for word in words:
        if word in word_prob_df.index:
            # Retrieve P(E|S) and P(E|¬S) from word_prob_df
            prob_word_given_spam = word_prob_df.loc[word, 'prob_spam']
            prob_word_given_ham = word_prob_df.loc[word, 'prob_ham']
            
            prob_spam.append(prob_word_given_spam)
            prob_not_spam.append(prob_word_given_ham)
    
    # Calculate the product of probabilities for spam and not spam
    prod_prob_spam = prior_spam * np.prod(prob_spam)
    prod_prob_not_spam = prior_ham * np.prod(prob_not_spam)
    
    if prod_prob_spam > prod_prob_not_spam:
        return 'spam'
    else:
        return 'ham'

# Example message to check
message_to_check = "Get your free offer now!"

# Calculate the spamliness of the message
classification = calculate_message_spamliness(message_to_check)

# Output the classification of the message
print(f"The message '{message_to_check}' is classified as: {classification}")



<h3>9. Avoiding floating point underflow</h3>
<p>Our aim is to compare two probabilities $P(S|x_1,\dots,x_n)$ with $P(\neg S|x_1,\dots,x_n),$ according to our model introduced in Section 8, both probabilities share a common denominator which does not affect comparison. Hence we will calculate numerators only, which are proportional to $P(S|x_1,\dots,x_n)$ and $P(\neg S|x_1,\dots,x_n).$
</p>

<p>Multiplying a set of small probabilities could result in a floating-point error.  This is where the product becomes too small to be represented correctly.  To avoid this we can take the logarithm of the probabilities and add them.  

To avoid multiplication of small numbers, we use the following property of $\log(x):$</p>
$$
\log(a\cdot b)=\log(a)+\log(b)
$$
<p>i.e. the log of the product is equal to the sum of logs (so instead of multiplying small numbers we will add them):</p>
$$
P(S|x_1,x_2,\dots,x_n)\propto P(S)\cdot P(x_1|S)\cdot \dots \cdot P(x_n|S)$$
<p>becomes</p>
$$\log(P(S|x_1,x_2,\dots,x_n))\propto \log\left(P(S)\cdot P(x_1|S)\cdot \dots  P(x_n|S)\right)=$$ $$
\log(P(S))+\log(P(x_1|S))+\dots+\log(P(x_n|S))
$$
<p>So, to check spam or ham we just compare:</p>
$$
\log(P(S))+\log(P(x_1|S))+\dots+\log(P(x_n|S))
$$
<p>and </p>
$$
\log(P(\neg S))+\log(P(x_1|\neg S))+\dots+\log(P(x_n|\neg S))
$$


Change the equation so that logs are used.
</p>


So, for comparing, we evaluate:

log(P(S)) + log(P(x1|S)) + ... + log(P(xn|S))

and

log(P(¬S)) + log(P(x1|¬S)) + ... + log(P(xn|¬S))

These equations represent the comparison of probabilities using logarithms, where log denotes the natural logarithm.

<h3>10. Testing the Model</h3>
<p>Now that we have tested the model using simple messages.  Let’s test the model using the messages from the test_set.  You should implement counters that displays how your model has performed and calculate the accuracy of the model.</p>
<pre>
match_spam 173
match_ham 843
thought_ham_is_spam 3
thought_spam_is_ham 357
Accuracy: 0.7383720930232558
</pre>



In [ ]:
match_spam = 0
match_ham = 0
thought_ham_is_spam = 0
thought_spam_is_ham = 0

for index, row in test_data.iterrows():
    message = row['v2']
    label = row['v1']
    
    classification = calculate_message_spamliness(message)
    
    if classification == 'spam' and label == 'spam':
        match_spam += 1
    elif classification == 'ham' and label == 'ham':
        match_ham += 1
    elif classification == 'ham' and label == 'spam':
        thought_ham_is_spam += 1
    elif classification == 'spam' and label == 'ham':
        thought_spam_is_ham += 1

total_messages = len(test_data)
accuracy = (match_spam + match_ham) / total_messages

print("match_spam:", match_spam)
print("match_ham:", match_ham)
print("thought_ham_is_spam:", thought_ham_is_spam)
print("thought_spam_is_ham:", thought_spam_is_ham)
print("Accuracy:", accuracy)


<h3>11. Improvements</h3>
<p>Utilise the validation set to assess the performance of various word sets in classifying spam and non-spam (ham) emails. Compare the effectiveness of different sets of words to determine their impact on classification accuracy.</p>


<h3></h3>

In [ ]:

all_words = set(word_freq_df.index)

top_words = set(word_freq_df.sort_values(by='spam_count', ascending=False).head(1000).index)

def calculate_message_spamliness_with_word_set(message, word_set):
    prob_spam = []
    prob_not_spam = []
    
    for word in message.split():
        if word in word_set:
            # Retrieve P(E|S) and P(E|¬S) from word_prob_df
            prob_word_given_spam = word_prob_df.loc[word, 'prob_spam']
            prob_word_given_ham = word_prob_df.loc[word, 'prob_ham']
            
            prob_spam.append(prob_word_given_spam)
            prob_not_spam.append(prob_word_given_ham)
    
    prod_prob_spam = prior_spam + np.sum(np.log(prob_spam))
    prod_prob_not_spam = prior_ham + np.sum(np.log(prob_not_spam))
    
    if prod_prob_spam > prod_prob_not_spam:
        return 'spam'
    else:
        return 'ham'

def evaluate_word_set(word_set):
    correct_count = 0
    total_count = 0
    
    for index, row in validation_data.iterrows():
        message = row['v2']
        label = row['v1']
        
        classification = calculate_message_spamliness_with_word_set(message, word_set)
        
        if classification == label:
            correct_count += 1
        total_count += 1
    
    accuracy = correct_count / total_count
    return accuracy

all_words_accuracy = evaluate_word_set(all_words)
top_words_accuracy = evaluate_word_set(top_words)

print("Accuracy with all words:", all_words_accuracy)
print("Accuracy with top words:", top_words_accuracy)